In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.core.display import display, HTML

In [2]:
first_html = 'https://www.realtor.com/realestateandhomes-detail/6545-SE-Yamhill-Ct_Portland_OR_97215_M12879-48012'
response = requests.get(first_html, headers={'User-Agent': 'Mozilla/5.0'})
page = response.text
soup = BeautifulSoup(page, "html5lib")

In [3]:
response.status_code

200

In [4]:
#Take into account all Portland zipcodes
def get_zipcodes(zipcode1, zipcode2):
    url_list = []
    for zip_code in range(zipcode1, zipcode2 + 1):    
        url = "https://www.redfin.com/zipcode/" + str(zip_code) + "/filter/include=sold-1mo"
        url_list.append(url)
    return url_list

zipcode_url_list = get_zipcodes(97201, 97266)
zipcode_url_list[0]

'https://www.redfin.com/zipcode/97201/filter/include=sold-1mo'

In [6]:
chromedriver = '/Applications/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver
test_zip = zipcode_url_list[11]
driver = webdriver.Chrome(chromedriver)
driver.get(test_zip)
driver.page_source[:1000]
test_soup = BeautifulSoup(driver.page_source, 'html.parser')
direct_house_links = []
all_links_on_page = test_soup.find_all(class_ = "slider-item")
for link in all_links_on_page:
    house_link_text = link['href']
    direct_house_links.append(house_link_text)

test_soup.find(class_= "home summary")


AttributeError: 'NoneType' object has no attribute 'text'

In [14]:
num_homes = int(test_soup.find(class_= "homes summary").text.split()[1])
if num_homes > 40:
    page2_url = zipcode + 'page-2'
    driver.get(zipcode)
    test_soup = BeautifulSoup(driver.page_source, 'html.parser') 

if num_homes > 80:
    page3_url = 

'6'

In [302]:
# Get links for sold homes in all zipcodes, put in error handling
def get_links(zipcode_list):
    house_html_list = []
    num_search_pages = []
    
    #Start up selenium
    chromedriver = '/Applications/chromedriver'
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    for zipcode in zipcode_list:
        print(zipcode)
        driver.get(zipcode)
        test_soup = BeautifulSoup(driver.page_source, 'html.parser') 
        
    #Determine number of houses so we can go through multiple pages
    
    num_houses_in_search = test_soup.find(class_= "home summary").text
    #Parse the page
    all_links_on_page = test_soup.find_all(class_ = "slider-item")
    for link in all_links_on_page:
        house_link_text = link['href']
        house_html_list.append(house_link_text)
    
    if #num_house > 41
        page2_url = zipcode + 'page-2'
        driver.get(page2_url)
        p2_soup = BeautifulSoup(driver.page_source, 'html.parser') 
        
        all_links_on_page = test_soup.find_all(class_ = "slider-item")
        for link in all_links_on_page:
            house_link_text = link['href']
            house_html_list.append(house_link_text)
    
    if #num_house > 82
        page3_url = zipcode + 'page-3'
        driver.get(page3_url)
        p3_soup = BeautifulSoup(driver.page_source, 'html.parser') 

        
    print(zipcode)
    time.sleep(.5+2*random.random())

all_house_links = get_links(zipcode_url_list)
len(all_house_links)

https://www.redfin.com/zipcode/97201/filter/include=sold-1mo


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=90.0.4430.93)


In [303]:
len(all_house_links)

NameError: name 'all_house_links' is not defined

In [ ]:
def get_house_dict(link):
    base_url = 
    url = base_url + link
    
    #Request HTML 
    response = requests.get(url)
        if response.status_code != 200:
            return None
        page = response.text
        soup = BeautifulSoup(page,"lxml")
    
    #Get address
        street = soup.find(itemprop="streetAddress")[0].text
        city = soup.find_all(itemprop="addressLocality")[0].text
        region = soup.find(itemprop="addressRegion")[0].text
        zipcode = soup.find(itemprop="postalCode")[0].text

    #Create house dictionary and return
        house_dict = dict(zip(headers, [street, city, region, zipcode])
        
        return house_dict

# sold_house_dict_list = 
# for link in sold_house.link_stub:
#     sold_house_dict_list.append(get_house_dict(link))


In [ ]:
def scrape_all(html_list):
    ua = UserAgent()
    for i, page in enumerate(page_list):
        user_agent = {'User-agent': ua.random}
        response = requests.get(html, headers=user_agent)
        if response.status_code != 200:
                    return None
        page = response.text
        soup = BeautifulSoup(page, "html5lib")
        headers = ['Beds', 'Bathrooms', 'Square Feet', 'Lot Size']
        
        time.sleep(.5+2*random.random())
        print(page)

In [163]:
headers = ['street', 'city', 'region', 'zip_Code']
address_dict = {}
address_soup = soup.find_all(itemprop="streetAddress")[0]
address = address_soup.text
city_soup = soup.find_all(itemprop="addressLocality")[0]
city = city_soup.text
region = region_soup.text
zip_soup = soup.find_all(itemprop="postalCode")[0]
zipcode = zip_soup.text

In [169]:
headers_list = ['Beds', 'Bathrooms', 'Square Feet', 'Lot Size']
gen_info_soup = soup.find_all(class_="data-value")
beds = int(gen_info_soup[0].text)
bathrooms = float(gen_info_soup[1].text.replace('+', ''))
sq_ft = int(gen_info_soup[2].text.replace(',',''))
sqft_lot = int(gen_info_soup[3].text.replace(',',''))
history_soup = 
year_built = 
year_renovated = 


(4, 3.5)

In [178]:
tables = pd.read_html(page)
price_history = tables[1]
listed_price = price_history[price_history['Event'] == 'Listed'].reset_index()
listed_price.iloc(0)

In [172]:
tables[0]
tables[0]['Rating']

,Rating*,School Name,Grades,Distance
0,8,Glencoe Elementary School,K–5,0.7 mi
1,4,Mt Tabor Middle School,6–8,0.5 mi
2,5,Franklin High School,9–12,1.1 mi
3,9,Portland Arthur Academy Charter School,K–5,0.5 mi
4,4,Bridger Elementary School,K–8,0.8 mi
5,4,Madison High School,9–12,1.9 mi
6,NR,Gately Academy Private School,4–8,0.9 mi
7,NR,The Portland Montessori Private School,PK–5,1.0 mi


In [102]:
address = soup.find_all(id = 'ds-chip-property-address')
for element in address:
    print(element.prettify())
street = address[0].text.replace('xa0','')
street

<h1 class="Text-c11n-8-33-0__aiai24-0 StyledHeading-c11n-8-33-0__ktujwe-0 lfNwrZ" font-family="sansSerif" id="ds-chip-property-address">
 <span>
  1527 NW Miller Rd,
 </span>
 <span>
  <!-- -->
  Portland, OR 97229
 </span>
</h1>


'1527 NW Miller Rd,\xa0Portland, OR 97229'

In [131]:
house_info = soup.find_all(class_="Text-c11n-8-33-0__aiai24-0 kTUCqv")
# for element in house_info:
#     print(element.prettify())

print(house_info[4].text)
i = 1
house_info = {}
while i in range(4,10):
    all_info = house_info[i].text
    info = all_info.split()[0]
    num = all_info.split()[1]
    i += 1

Bedrooms: 4
